In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-154251
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-154251


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_name = "train-compute"
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, compute_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)



InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({"--C": choice(0.001, 0.01, 0.1, 1, 10,20,50,100,200,500,1000 ),
                                "--max_iter": choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=2)
if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
shutil.copy('./train.py', script_folder)    

# Create a SKLearn estimator for use with train.py
est =SKLearn(source_directory=script_folder,compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4,
    estimator=est,
    policy=policy

)

In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e2ab185a-4526-4556-b4f8-2c306c634e2a
Web View: https://ml.azure.com/runs/HD_e2ab185a-4526-4556-b4f8-2c306c634e2a?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-153928/workspaces/quick-starts-ws-153928&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-08T05:45:44.943166][API][INFO]Experiment created<END>\n""<START>[2021-08-08T05:45:45.910943][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-08-08T05:45:45.684007][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_e2ab185a-4526-4556-b4f8-2c306c634e2a
Web View: https://ml.azure.com/runs/HD_e2ab185a-4526-4556-b4f8-2c306c634e2a?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-153928/workspaces/quick-starts-ws-153928&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_e2ab185a-4526-4556-b4f8-2c306c634e2a',
 'target': 'train-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-08-08T05:45:44.713623Z',
 'endTimeUtc': '2021-08-08T05:51:50.628975Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'fb288005-f521-4e70-96d9-f58732bc5dd6',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.9072837632776934',
  'best_child_run_id': 'HD_e2ab185a-4526-4556-b4f8-2c306c634e2a_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg153928.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_e2ab185a-4526-4556-b4f8-2c306c634e2a/azureml-logs/hyperdrive.txt?sv=2019-02-02&sr=b

In [14]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_file_names())
hyperdrive_model = best_run.register_model(model_name='best-run-model', model_path = 'outputs/model.joblib')
print(best_run)

['azureml-logs/55_azureml-execution-tvmps_4a1dc745d2a74b4b493b4cc985666f1d5eb5d223ff5b5ad8f4544bf9f9ce160d_d.txt', 'azureml-logs/65_job_prep-tvmps_4a1dc745d2a74b4b493b4cc985666f1d5eb5d223ff5b5ad8f4544bf9f9ce160d_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_4a1dc745d2a74b4b493b4cc985666f1d5eb5d223ff5b5ad8f4544bf9f9ce160d_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Run(Experiment: udacity-project,
Id: HD_e2ab185a-4526-4556-b4f8-2c306c634e2a_0,
Type: azureml.scriptrun,
Status: Completed)


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(url)


In [4]:
from train import clean_data
from sklearn.preprocessing import OneHotEncoder
import pandas as pd 
from azureml.core.run import Run
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x,y = clean_data(ds)
y_df = pd.DataFrame(y, columns=['y'])
if not os.path.isdir('training'):
    os.mkdir('training')
data = pd.concat([x,y_df], axis=1)
x_train, x_test = train_test_split(data, test_size=0.2, random_state = 1)
x_train.to_csv("training/train_data.csv", index=False)
dataset = ws.get_default_datastore()
dataset.upload(src_dir='./training', target_path='udacity-project', overwrite=True, show_progress=True)
train_data = Dataset.Tabular.from_delimited_files(path=dataset.path('udacity-project/train_data.csv'))


#if not os.path.isdir('data'):
    #os.mkdir('data')
#pd.DataFrame(ds).to_csv("data/train_data.csv", index=False)
#dataset = ws.get_default_datastore()
#dataset.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)

#train_data = Dataset.Tabular.from_delimited_files(path=dataset.path('bankmarketing/train_data.csv'))






Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [5]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data=train_data,
    n_cross_validations= 5,
    label_column_name = 'y',
    compute_target=cpu_cluster,
    enable_onnx_compatible_models=True)

In [6]:
# Submit your automl run
from azureml.widgets import RunDetails

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

Submitting remote run.
No run_configuration provided, running on train-compute with default configuration
Running on remote compute: train-compute


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_927aad81-aecf-4130-849b-24fac2762cde,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [10]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, model = remote_run.get_output()
print(best_automl_run)
best_automl_run.register_model(model_name='automl.pkl', model_path='./outputs')
print(model._final_estimator)

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
Package:azureml-automl-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-core, training version:1.33.0, current version:1.32.0
Package:azureml-dataprep, training version:2.20.1, current version:2.18.0
Package:azureml-dataprep-native, training version:38.0.0, current version:36.0.0
Package:azureml-dataprep-rslex, training version:1.18.0, current version:1.16.1
Package:azureml-dataset-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-defaults, training version:1.33.0, current version:1.32.0
Package:azureml-interpret, training version:1.33.0, current version:1.32.0
Package:azureml-mlflow, training version:1.33.0, current version:1.32.0
Package:azureml-pipeline-core, training version:1.33.0, current version:1.32.0
Package:azureml-responsibleai, training version:1.33.0, current version:1.32.0
Package:azureml-telemetry, training version:1.33.0, current version:1

Run(Experiment: udacity-project,
Id: AutoML_927aad81-aecf-4130-849b-24fac2762cde_25,
Type: azureml.scriptrun,
Status: Completed)
PreFittedSoftVotingClassifier(
    estimators=[('0', Pipeline(memory=None,
             steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                    ('lightgbmclassifier',
                     LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(
        dataset_samples=26360,
        dataset_features=122,
        dataset_classes=None,
        dataset_num_categorical=0,
        dataset_categoricals=None,
        pipeline_categoricals=None,
        dataset_y_std=None,
        dataset_uid=None,
        subsamp...
        subsampling_schedule='hyperband_clip',
        cost_mode_param=None,
        iteration_timeout_mode=0,
        iteration_timeout_param=None,
        feature_column_names=None,
        label_column_name=None,
        weight_column_name=None,
        cv_split_column_names=None,
        enable_streaming=None,
       

In [11]:
cpu_cluster.delete()

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
